# Active Learning dashboard

In [1]:
import json
import psycopg2
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

with open('config.json') as f:
    conf = json.load(f)
conn_str = "host={} dbname={} user={} password={}".format(conf['host'], conf['database'], conf['user'], conf['passw'])
conn = psycopg2.connect(conn_str)
plotly.tools.set_credentials_file(username=conf['plotlyusername'], api_key=conf['plotlyapikey'])

/Users/saakhta/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Plotting a pie chart for current image tagging status:

In [2]:
names = []
numbers = []
cursor = conn.cursor()
cursor.execute('select count(a.imageid), b.tagstatename from Image_Tagging_State a join tag_state b ON a.tagstateid = b.tagstateid group by b.tagstatename')
for (number, name) in cursor:
    names.append(name)
    numbers.append(number)

fig = {
    'data': [{'labels': names,
              'values': numbers,
              'type': 'pie'}],
    'layout': {'title': 'Tag state by count'}
     }

py.iplot(fig)

Plotting a time chart for the tagging activity over the last few days: 

In [3]:
dates = []
numimages = []
cursor = conn.cursor()
cursor.execute("select count(imageid), date_trunc('day', modifieddtim) timerange from image_tagging_state_audit group by date_trunc('day', modifieddtim) order by timerange")
for (numimage, date) in cursor:
    x = pd.to_datetime(date)
    dates.append(x)
    numimages.append(numimage)

# Create a trace
trace = go.Scatter(
    x = dates,
    y = numimages,
    # mode = 'markers'
    name = 'Number of tagging activities'
)
data = [trace]
layout = dict(title = 'Number of tagging activities by date', xaxis=dict(title='Date'), yaxis=dict(title='Tagging activities'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='basic-scatter')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~samiyaakhtar/0 or inside your plot.ly account where it is named 'basic-scatter'


Top taggers 

In [4]:
cursor = conn.cursor()
usernames = []
tagcounts = []
cursor.execute("SELECT e.username, count(*) as TagCount FROM image_info a join Annotated_Labels b on a.imageid = b.imageid join user_info e on b.createdbyuser = e.userid group by e.username order by TagCount desc")
for (username, tagcount) in cursor: 
    usernames.append(username)
    tagcounts.append(tagcount)

fig = {
    'data': [{'labels': usernames,
              'values': tagcounts,
              'type': 'pie'}],
    'layout': {'title': 'Top taggers by number of classifications'}
     }

py.iplot(fig)

In [5]:
class_data = {}
cursor = conn.cursor()
cursor.execute("SELECT e.username, d.classificationname, count(*) as TagCount FROM image_info a join Annotated_Labels b on a.imageid = b.imageid join Annotated_Labels c on b.imagetagid = c.imagetagid join classification_info d on c.classificationid = d.classificationid  join user_info e on b.createdbyuser = e.userid group by e.username, d.classificationname order by TagCount desc")
for (username, classname, tagcount) in cursor:
    if username not in class_data:
        class_data[username] = {}
        class_data[username]['classnames'] = []
        class_data[username]['tagcount'] = []
    class_data[username]['classnames'].append(classname)
    class_data[username]['tagcount'].append(tagcount)
    
data = []
for key in class_data:
    trace = go.Bar(
        x=class_data[key]['classnames'],
        y=class_data[key]['tagcount'],
        name=key
    )
    data.append(trace)

layout = go.Layout(
    barmode='stack',
    title='Top taggers by classification info'
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')


In [6]:
cursor = conn.cursor()
cursor.execute('SELECT b.classificationname, count(*) AS ClassificationCount FROM Annotated_Labels a join classification_info b on a.classificationid = b.classificationid group by classificationname order by ClassificationCount desc')
classnames = []
counts = []
for (classname, count) in cursor:
    classnames.append(classname)
    counts.append(count)

trace = go.Bar(
    x=classnames,
    y=counts
)
data = [trace]
layout = dict(title = 'Top classifications', xaxis=dict(title='Classification'), yaxis=dict(title='Number of tags'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='basic-plot')

In [7]:
cursor = conn.cursor()
cursor.execute('SELECT p.imageid, c.classificationname,p.boxconfidence, p.imageconfidence FROM Prediction_Labels p join classification_info c on c.classificationid = p.classificationid where trainingid = (SELECT MAX(trainingid) FROM training_info) order by boxconfidence ASC;')
classnames = []
counts = []
boxconfs = []
imageconfs = []
for (imageid, classname, boxconf, imageconf) in cursor:
    classnames.append(classname)
    boxconfs.append(boxconf)
    imageconfs.append(imageconf)

trace1 = go.Bar(
    x=classnames,
    y=boxconfs
)

trace2 = go.Bar(
    x=classnames,
    y=imageconfs
)
data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    title = 'Least confident classifications', 
    xaxis=dict(title='Box confidence and Image confidence'), 
    yaxis=dict(title='Classnames')
)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='grouped-plot')

In [8]:
cursor = conn.cursor()
cursor.execute('SELECT p.imageid, c.classificationname,p.boxconfidence, p.imageconfidence FROM Prediction_Labels p join classification_info c on c.classificationid = p.classificationid where trainingid = (SELECT MAX(trainingid) FROM training_info) order by boxconfidence DESC;')
classnames = []
counts = []
boxconfs = []
imageconfs = []
for (imageid, classname, boxconf, imageconf) in cursor:
    classnames.append(classname)
    boxconfs.append(boxconf)
    imageconfs.append(imageconf)

trace1 = go.Bar(
    x=classnames,
    y=boxconfs
)

trace2 = go.Bar(
    x=classnames,
    y=imageconfs
)
data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    title = 'Most confident classifications', 
    xaxis=dict(title='Box confidence and Image confidence'), 
    yaxis=dict(title='Classnames')
)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='grouped-plot')

In [10]:
cursor = conn.cursor()
cursor.execute("SELECT count(c.classificationname), c.classificationname, date_trunc('day', t.createddtim) timerange FROM training_info t  join class_performance p on t.trainingid = p.trainingid join classification_info c on c.classificationid = p.classificationid group by c.classificationname, timerange order by timerange desc;")
classnames = []
counts = []
dates = []
for (count, classname, time) in cursor:
    classnames.append(classname)
    counts.append(count)
    dates.append(time)
    

# Create a trace
trace = go.Scatter(
    x = dates,
    y = counts,
    # mode = 'markers'
    name = 'Class performance over time'
)
data = [trace]
layout = dict(title = 'Class performance over time', xaxis=dict(title='Date'), yaxis=dict(title='Class performance'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='basic-scatter')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~samiyaakhtar/0 or inside your plot.ly account where it is named 'basic-scatter'
